In [7]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm
import nltk
import re
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import zero_one_loss
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import t
from nltk.probability import FreqDist
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mislavz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mislavz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mislavz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
def nested_kfold_cv(clf, param_grid, X, y, k1=10, k2=3):
    
    err = []
    kfold = KFold(n_splits=k1, shuffle=True, random_state=42)
    
    # Outer loop
    for ind_train, ind_test in kfold.split(X):
        
        X_train, y_train, X_test, y_test = X[ind_train], y[ind_train], X[ind_test], y[ind_test]
        
        # Inner loop
        inn = GridSearchCV(clf,param_grid,n_jobs=5, cv=StratifiedKFold(n_splits=5, 
                                              random_state=42).split(X_train, y_train), 
                           verbose=2).fit(X_train, y_train)
        
        # Prediction based on the best selected params, the ones that minimize average error
        h = inn.best_estimator_.fit(X_train, y_train).predict(X_test)
        
        err.append(zero_one_loss(y_test, h))
        print(classification_report( y_test, h ))
    return err

In [10]:

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [11]:
def get_common_words(content):
    ret_list = []
    fdist2 = nltk.FreqDist(content)
    most_list = fdist2.most_common(75)
    for x in most_list:
        ret_list.append(x)
    return ret_list

In [12]:

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)

#Text cleaning and tokenization, then stemming then POS tagging
filtered_tweets=[];
tweet_tags = [];
filtered_tweets_stemmed=[];
common_words_prepare=[];

for tweet in tweets:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    common_words = get_common_words(word_tokens)
    for word in common_words:
        common_words_prepare.append(word)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in filtered_tweets:
    tweet_tags.append(nltk.pos_tag(tweet))
    


In [13]:
# Izbacivanje frequent wordsa

common_words = get_common_words(common_words_prepare)
#for word in filtered_tweets[1:100]:
#    print(word)
common = []
for word in common_words[1:50]:
   common.append(word[0][0])

filtered_tweets_no_common = [];
filtered_tweets_no_common_stemmed = [];
for line in filtered_tweets:
    filtered_tweets_no_common.append([word for word in line if not word in common])
    filtered_tweets_no_common_stemmed.append([ps.stem(word) for word in line if not word in common])
#print("---------------------------")
#for word in filtered_tweets_no_common[1:100]:
#    print(word)
    

In [18]:
#Creating TfIdf vectorizer
ftss=[]
for tweet in filtered_tweets_no_common_stemmed:
    ftss.append(' '.join(tweet))

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),lowercase=False,max_features=1000,smooth_idf=False,norm=None,max_df=0.75,min_df=5)



In [19]:
X = vectorizer.fit_transform(ftss).toarray()
Y = dataset['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3)



In [15]:
#SVM Classifier
C = [ 2e-2, 2e-1, 2e-0]
gama = [ 2e-2, 2e-1, 2e-0]
param = [{'svc__kernel': ['rbf'], 'svc__C': C}]
clf = svm.SVC(decision_function_shape='ovo')
scale = StandardScaler()
pipeline = Pipeline([('scaler', scale), ('svc', clf)])

svm_err = nested_kfold_cv(pipeline, param, X, Y)
#param_grid = [{}] 
#grid_search = GridSearchCV(pipeline, 
#                           param_grid,
#                          n_jobs = 5,
#                           cv=StratifiedKFold(n_splits=5, 
#                                              random_state=42).split(X_train, y_train), 
#                           verbose=2)
#model = grid_search.fit(X_train,y_train)
#y_pred = model.predict(X_test)
#report = classification_report( y_test, y_pred )
#print(report)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 69.7min finished


             precision    recall  f1-score   support

          0       0.48      0.12      0.19       164
          1       0.84      0.97      0.90      1905
          2       0.79      0.44      0.56       410

avg / total       0.80      0.82      0.80      2479

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 69.8min finished


             precision    recall  f1-score   support

          0       0.38      0.16      0.22       127
          1       0.84      0.97      0.90      1927
          2       0.82      0.42      0.56       425

avg / total       0.82      0.83      0.81      2479

Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

             precision    recall  f1-score   support

          0       0.45      0.13      0.21       135
          1       0.84      0.97      0.90      1907
          2       0.82      0.45      0.58       436

avg / total       0.82      0.83      0.81      2478

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 72.9min finished


             precision    recall  f1-score   support

          0       0.63      0.13      0.21       133
          1       0.86      0.97      0.91      1937
          2       0.81      0.49      0.61       408

avg / total       0.84      0.85      0.82      2478

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 68.5min finished


             precision    recall  f1-score   support

          0       0.42      0.13      0.20       124
          1       0.86      0.98      0.91      1942
          2       0.86      0.50      0.63       412

avg / total       0.84      0.85      0.83      2478



In [14]:
#Logistic Regression

C = [2e-3, 2e-2, 2e-1, 2e-0, 2e-1, 2e-2, 2e-3]

std_scaler = StandardScaler()
clf = LogisticRegression(multi_class='ovr',solver='newton-cg')
pipeline = Pipeline([('scaler', std_scaler)  , ('clf', clf)])
param = [{'clf__C': C}]

logreg_err = nested_kfold_cv(pipeline, param, X, Y)



#TODO: logicstic regression, model selection, kfold

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.53      0.10      0.17       164
          1       0.85      0.97      0.90      1905
          2       0.80      0.54      0.65       410

avg / total       0.82      0.84      0.81      2479

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.47      0.12      0.19       127
          1       0.86      0.97      0.91      1927
          2       0.80      0.50      0.62       425

avg / total       0.83      0.85      0.82      2479

Fitting 5 folds for each of 7 candidates, totalling 35 fits


...

             precision    recall  f1-score   support

          0       0.41      0.11      0.17       135
          1       0.85      0.96      0.90      1907
          2       0.80      0.55      0.65       436

avg / total       0.82      0.84      0.82      2478

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.52      0.11      0.17       133
          1       0.86      0.97      0.91      1937
          2       0.79      0.55      0.65       408

avg / total       0.83      0.85      0.83      2478

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.57      0.17      0.26       124
          1       0.88      0.98      0.92      1942
          2       0.86      0.58      0.70       412

avg / total       0.86      0.87      0.85      2478



In [8]:
from scipy.stats import ttest_rel as paired_t_test

t_stat, p_val = paired_t_test(logreg_err, svm_err)

if p_val <= 0.05:
    print('\np-value = ', p_val, ', hipoteza se odbacuje.')
else:
    print('\np-value = ', p_val, ', hipoteza se ne odbacuje.')

NameError: name 'logreg_err' is not defined

In [14]:
#Sampling data so we get equal distribution for all classes. used in active learning
d0 = dataset[dataset['class'] == 0]
d1 = dataset[dataset['class'] == 1].sample(d0.shape[0])
d2 = dataset[dataset['class'] == 2].sample(d0.shape[0])

#Dl data that is representing same number of all classes. From this we will take n data for starter training as
#labeled data, and rest for unlabeled data for active learning
Dl = pd.concat([d0,d1,d2])
Dl = sklearn.utils.shuffle(Dl)
Dl.reset_index(drop=True)





,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,20433,3,0,1,2,2,"RT @seanmdav: Since I know you're wondering, @..."
1,8014,6,3,1,2,0,Apaches' 30 MM rounds turn Jihadis into tomato...
2,14151,3,0,3,0,1,Peter Pan that ho never land that ho
3,3777,3,3,0,0,0,@JoeDouglas1 sit down and stfu raghead @Darkni...
4,8907,3,0,3,0,1,"Day juan at the Jim, and Im in total bitch mod..."
5,14052,3,0,0,3,2,Or pollo
6,15348,3,1,2,0,1,RT @Dhet_Btchh: Hoes really be having built up...
7,15905,3,0,3,0,1,RT @HuhWhatsACondom: BRUUUUUUH &#128557;&#1285...
8,17295,3,0,1,2,2,"RT @PlMPCESS: In the fashion world ""urban"" def..."
9,21119,3,2,1,0,0,Seeing ugly bitches get pregnant let's me know...


In [15]:
AL_filtered_tweets=[];
AL_tweet_tags = [];
AL_filtered_tweets_stemmed=[];
AL_common_words_prepare=[];

for tweet in Dl['tweet']:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    common_words = get_common_words(word_tokens)
    for word in common_words:
        AL_common_words_prepare.append(word)
    AL_filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    AL_filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in AL_filtered_tweets:
    AL_tweet_tags.append(nltk.pos_tag(tweet))

# Izbacivanje frequent wordsa

AL_common_words = get_common_words(AL_common_words_prepare)
#for word in filtered_tweets[1:100]:
#    print(word)
AL_common = []
for word in AL_common_words[1:50]:
   AL_common.append(word[0][0])

AL_filtered_tweets_no_common = [];
AL_filtered_tweets_no_common_stemmed = [];
for line in AL_filtered_tweets:
    AL_filtered_tweets_no_common.append([word for word in line if not word in AL_common])
    AL_filtered_tweets_no_common_stemmed.append([ps.stem(word) for word in line if not word in AL_common])
#print("---------------------------")
#for word in filtered_tweets_no_common[1:100]:
#    print(word)

AL_ftss=[]
for tweet in AL_filtered_tweets_no_common_stemmed:
    AL_ftss.append(' '.join(tweet))


In [20]:
AL_X = vectorizer.fit_transform(AL_ftss).toarray()
AL_Y = Dl['class'].tolist()

In [21]:
import math
Tk_X, Tu_X, Tk_y, _ = train_test_split(AL_X, AL_Y, random_state=42, test_size=0.8)

n_labeled_points = math.ceil(len(AL_Y)*0.05)
unlabeled_indices = np.arange(len(AL_Y))[n_labeled_points:]


In [22]:
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse.csgraph import connected_components
import matplotlib.pyplot as plt
from scipy import stats
max_iterations = 100
for i in range(max_iterations):
    if len(Tu_X) == 0 :
        print("No unlabeled data to label.")
        break
    al_model = label_propagation.LabelSpreading(gamma=0.25,max_iter=200)
    y_train = np.copy(AL_Y)
    y_train[unlabeled_indices] = -1
    al_model.fit(AL_X,y_train)
    predicted_labels = al_model.transduction_[unlabeled_indices]
    true_labels = [AL_Y[i] for i in unlabeled_indices]
    #if set(al_model.classes_).issubset(set(true_labels)) == False:
    #    break
    #cm = confusion_matrix(true_labels, predicted_labels,
    #                      labels=al_model.classes_)
    #print("Iteration %i %s" % (i, 70 * "_"))
    
        
    #print("Label Spreading model: %d labeled & %d unlabeled (%d total)"
    #      % (n_labeled_points, len(AL_Y) - n_labeled_points,
    #         len(AL_Y)))

    #print(classification_report(true_labels, predicted_labels))

    #print("Confusion matrix")
    #print(cm)

    # compute the entropies of transduced label distributions
    pred_entropies = stats.distributions.entropy(
        al_model.label_distributions_.T)

    # select up to 5 digit examples that the classifier is most uncertain about
    uncertainty_index = np.argsort(pred_entropies)[::-1]
    uncertainty_index = uncertainty_index[
        np.in1d(uncertainty_index, unlabeled_indices)][:50]

    # keep track of indices that we get labels for
    delete_indices = np.array([])
    
    for index, check_index in enumerate(uncertainty_index):
        check = AL_ftss[check_index]

        delete_index, = np.where(unlabeled_indices == check_index)
        delete_indices = np.concatenate((delete_indices, delete_index))

    unlabeled_indices = np.delete(unlabeled_indices, delete_indices)
    n_labeled_points += len(uncertainty_index)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [23]:
al_err = []
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    
# Outer loop
for ind_train, ind_test in kfold.split(X):
        
    X_train, y_train, X_test, y_test = X[ind_train], Y[ind_train], X[ind_test], Y[ind_test]
    
    n_labeled_points = math.ceil(len(y_train)*0.05)
    unlabeled_indices = np.arange(len(y_train))[n_labeled_points:]
    
    max_iterations = 100
    for i in range(max_iterations):
        if len(Tu_X) == 0 :
            print("No unlabeled data to label.")
            break
        al_model = label_propagation.LabelSpreading(gamma=0.25,max_iter=200)
        yal_train = np.copy(y_train)
        yal_train[unlabeled_indices] = -1
        al_model.fit(X_train,yal_train)
    
        predicted_labels = al_model.transduction_[unlabeled_indices]
    
        true_labels = [y_train for i in unlabeled_indices]
    
        pred_entropies = stats.distributions.entropy(al_model.label_distributions_.T)   
        uncertainty_index = np.argsort(pred_entropies)[::-1]
        uncertainty_index = uncertainty_index[np.in1d(uncertainty_index, unlabeled_indices)][:50]
        delete_indices = np.array([])
    
        for index, check_index in enumerate(uncertainty_index):
            check = X[check_index]

            delete_index, = np.where(unlabeled_indices == check_index)
            delete_indices = np.concatenate((delete_indices, delete_index))

        unlabeled_indices = np.delete(unlabeled_indices, delete_indices)
        n_labeled_points += len(uncertainty_index)
    
    h = al_model.predict(X_test)
    
    al_err.append(zero_one_loss(y_test, h))
    
    print(classification_report( y_test, h ))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


KeyboardInterrupt: 